In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)


4.7.1


In [4]:
portfolio_df = pd.read_excel('Sample stocks acquisition dates_costs.xlsx', sheetname='Sample')

portfolio_df.head(10)

,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year
0,2013-02-07,AAPL,125,65.40,8175.00,2017-12-29
1,2014-02-27,JNJ,100,81.90,8190.00,2017-12-29
2,2013-06-04,MCD,100,84.99,8499.00,2017-12-29
3,2015-12-14,MTCH,600,13.63,8178.00,2017-12-29
4,2016-01-14,NFLX,75,108.71,8153.25,2017-12-29
5,2013-08-14,WMT,125,68.30,8537.50,2017-12-29
6,2013-12-13,FB,150,53.32,7998.00,2017-12-29
7,2015-01-05,TWTR,225,36.38,8185.50,2017-12-29


In [32]:
# The below will pull back stock prices from 2010 until end date specified.
start_sp = datetime.datetime(2013, 1, 1)
end_sp = datetime.datetime(2018, 3, 9)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2017, 12, 29)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2013, 1, 1)
stocks_end = datetime.datetime(2018, 3, 9)

In [16]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 

sp500 = pdr.get_data_yahoo('^GSPC', 
                           start_sp,
                             end_sp)
                          
sp500.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,1462.420044,4202600000
2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,1459.369995,3829730000
2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,1466.469971,3424290000
2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,1461.890015,3304970000
2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,1457.150024,3601600000


In [18]:
sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-05,2681.060059,2728.090088,2675.750000,2720.939941,2720.939941,3710810000
2018-03-06,2730.179932,2732.080078,2711.260010,2728.120117,2728.120117,3370690000
2018-03-07,2710.179932,2730.600098,2701.739990,2726.800049,2726.800049,3393270000
2018-03-08,2732.750000,2740.449951,2722.649902,2738.969971,2738.969971,3212320000
2018-03-09,2752.909912,2786.570068,2751.540039,2786.570068,2786.570068,3364100000


In [20]:
sp500_adj_close = sp500[['Adj Close']].reset_index()


In [21]:
sp500_adj_close.tail()


,Date,Adj Close
1301,2018-03-05,2720.939941
1302,2018-03-06,2728.120117
1303,2018-03-07,2726.800049
1304,2018-03-08,2738.969971
1305,2018-03-09,2786.570068


In [24]:
sp_500_adj_close_start = sp500_adj_close[sp500_adj_close['Date']==end_of_last_year]
sp_500_adj_close_start

,Date,Adj Close
1258,2017-12-29,2673.610107


In [25]:
tickers = portfolio_df['Ticker'].unique()
tickers

array(['AAPL', 'JNJ', 'MCD', 'MTCH', 'NFLX', 'WMT', 'FB', 'TWTR'],
      dtype=object)

In [26]:
def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))
               
all_data = get(tickers, stocks_start, stocks_end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [27]:
all_data.head()

Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
AAPL   2013-01-02  79.117142  79.285713  77.375717  78.432854  68.502106   
       2013-01-03  78.268570  78.524284  77.285713  77.442856  67.637436   
       2013-01-04  76.709999  76.947144  75.118568  75.285713  65.753418   
       2013-01-07  74.571426  75.614288  73.599998  74.842857  65.366646   
       2013-01-08  75.601425  75.984283  74.464287  75.044289  65.542587   

                      Volume  
Ticker Date                   
AAPL   2013-01-02  140129500  
       2013-01-03   88241300  
       2013-01-04  148583400  
       2013-01-07  121039100  
       2013-01-08  114676800

In [29]:
adj_close = all_data[['Adj Close']].reset_index()
adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,68.502106
1,AAPL,2013-01-03,67.637436
2,AAPL,2013-01-04,65.753418
3,AAPL,2013-01-07,65.366646
4,AAPL,2013-01-08,65.542587


In [30]:
adj_close_start = adj_close[adj_close['Date']==end_of_last_year]
adj_close_start.head()

,Ticker,Date,Adj Close
1258,AAPL,2017-12-29,163.427597
2564,JNJ,2017-12-29,130.557571
3870,MCD,2017-12-29,162.893448
4449,MTCH,2017-12-29,29.799627
5755,NFLX,2017-12-29,191.960007


In [33]:
adj_close_latest = adj_close[adj_close['Date']==stocks_end]
adj_close_latest

,Ticker,Date,Adj Close
1305,AAPL,2018-03-09,174.517639
2611,JNJ,2018-03-09,125.826385
3917,MCD,2018-03-09,149.752518
4496,MTCH,2018-03-09,42.629364
5802,NFLX,2018-03-09,331.440002
7108,WMT,2018-03-09,84.700142
8414,FB,2018-03-09,185.229996
9505,TWTR,2018-03-09,35.349998


In [34]:
adj_close_latest.set_index('Ticker', inplace=True)
adj_close_latest.head()


,Date,Adj Close
Ticker,,
AAPL,2018-03-09,174.517639
JNJ,2018-03-09,125.826385
MCD,2018-03-09,149.752518
MTCH,2018-03-09,42.629364
NFLX,2018-03-09,331.440002


In [35]:
portfolio_df.set_index(['Ticker'], inplace=True)

portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29
MCD,2013-06-04,100,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29


In [38]:
merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)
merged_portfolio.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close
Ticker,,,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385
MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002


In [39]:
merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1

merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
Ticker,,,,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342
MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846
WMT,2013-08-14,125,68.30,8537.50,2017-12-29,2018-03-09,84.700142,0.240119
FB,2013-12-13,150,53.32,7998.00,2017-12-29,2018-03-09,185.229996,2.473931
TWTR,2015-01-05,225,36.38,8185.50,2017-12-29,2018-03-09,35.349998,-0.028312


In [40]:

merged_portfolio.reset_index(inplace=True)

In [42]:
merged_portfolio_sp = pd.merge(merged_portfolio, sp500_adj_close, left_on='Acquisition Date', right_on='Date')
# .set_index('Ticker')

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date_x,Adj Close_x,ticker return,Date_y,Adj Close_y
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,2013-02-07,1509.390015
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,2014-02-27,1854.290039
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,2013-06-04,1631.380005
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2015-12-14,2021.939941
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,2016-01-14,1921.839966


In [43]:
del merged_portfolio_sp['Date_y']

merged_portfolio_sp.rename(columns={'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'SP 500 Initial Close'}, inplace=True)

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966


In [44]:
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,5.416095
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,4.416785
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,5.209700
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419


In [46]:
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp500_adj_close, left_on='Latest Date', right_on='Date')

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,5.416095,2018-03-09,2786.570068
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,4.416785,2018-03-09,2786.570068
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,5.209700,2018-03-09,2786.570068
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2018-03-09,2786.570068
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2018-03-09,2786.570068


In [47]:
del merged_portfolio_sp_latest['Date']

merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,5.416095,2786.570068
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,4.416785,2786.570068
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,5.209700,2786.570068
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2786.570068
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068


In [48]:
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['SP Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,5.416095,2786.570068,0.846156,0.822309,21814.704895,15092.328747,6722.376148,13639.704895,6917.328747
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,4.416785,2786.570068,0.502769,0.033572,12582.638550,12307.680233,274.958317,4392.638550,4117.680233
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,5.209700,2786.570068,0.708106,0.053896,14975.251770,14517.193382,458.058388,6476.251770,6018.193382
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2786.570068,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243


In [49]:
merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on='Ticker')
# , how='outer'

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,...,2786.570068,0.846156,0.822309,21814.704895,15092.328747,6722.376148,13639.704895,6917.328747,2017-12-29,163.427597
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,...,2786.570068,0.502769,0.033572,12582.638550,12307.680233,274.958317,4392.638550,4117.680233,2017-12-29,130.557571
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,...,2786.570068,0.708106,0.053896,14975.251770,14517.193382,458.058388,6476.251770,6018.193382,2017-12-29,162.893448
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,2786.570068,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,2017-12-29,29.799627
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,2786.570068,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,2017-12-29,191.960007


In [50]:
# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.

del merged_portfolio_sp_latest_YTD['Date']

merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,5.416095,2786.570068,0.846156,0.822309,21814.704895,15092.328747,6722.376148,13639.704895,6917.328747,163.427597
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,4.416785,2786.570068,0.502769,0.033572,12582.638550,12307.680233,274.958317,4392.638550,4117.680233,130.557571
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,5.209700,2786.570068,0.708106,0.053896,14975.251770,14517.193382,458.058388,6476.251770,6018.193382,162.893448
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2786.570068,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007


In [59]:

merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start
                                             , left_on='Start of Year', right_on='Date')

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,...,0.846156,0.822309,21814.704895,15092.328747,6722.376148,13639.704895,6917.328747,163.427597,2017-12-29,2673.610107
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,...,0.502769,0.033572,12582.638550,12307.680233,274.958317,4392.638550,4117.680233,130.557571,2017-12-29,2673.610107
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,...,0.708106,0.053896,14975.251770,14517.193382,458.058388,6476.251770,6018.193382,162.893448,2017-12-29,2673.610107
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627,2017-12-29,2673.610107
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007,2017-12-29,2673.610107


In [67]:
# Deleting another unneeded Date column.


# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'SP Start Year Close'}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['SP Start Year Close'] - 1

merged_portfolio_sp_latest_YTD_sp.head()


,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,...,0.822309,21814.704895,15092.328747,6722.376148,13639.704895,6917.328747,163.427597,2673.610107,0.067859,0.04225
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,...,0.033572,12582.638550,12307.680233,274.958317,4392.638550,4117.680233,130.557571,2673.610107,-0.036238,0.04225
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,...,0.053896,14975.251770,14517.193382,458.058388,6476.251770,6018.193382,162.893448,2673.610107,-0.080672,0.04225
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627,2673.610107,0.430533,0.04225
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007,2673.610107,0.726610,0.04225


In [68]:

merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by='Ticker', ascending=True)
merged_portfolio_sp_latest_YTD_sp

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,...,0.822309,21814.704895,15092.328747,6722.376148,13639.704895,6917.328747,163.427597,2673.610107,0.067859,0.04225
6,FB,2013-12-13,150,53.32,7998.00,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,1.904315,27784.499359,12553.786405,15230.712954,19786.499359,4555.786405,176.460007,2673.610107,0.049700,0.04225
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,...,0.033572,12582.638550,12307.680233,274.958317,4392.638550,4117.680233,130.557571,2673.610107,-0.036238,0.04225
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,...,0.053896,14975.251770,14517.193382,458.058388,6476.251770,6018.193382,162.893448,2673.610107,-0.080672,0.04225
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627,2673.610107,0.430533,0.04225
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007,2673.610107,0.726610,0.04225
7,TWTR,2015-01-05,225,36.38,8185.50,2017-12-29,2018-03-09,35.349998,-0.028312,2020.579956,...,-0.407406,7953.749657,11288.575454,-3334.825797,-231.750343,3103.075454,24.010000,2673.610107,0.472303,0.04225
5,WMT,2013-08-14,125,68.30,8537.50,2017-12-29,2018-03-09,84.700142,0.240119,1685.390015,...,-0.413249,10587.517738,14115.630063,-3528.112325,2050.017738,5578.130063,93.716957,2673.610107,-0.096213,0.04225


In [69]:

# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst']

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult
0,AAPL,2013-02-07,125,65.40,8175.0,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,...,13639.704895,6917.328747,163.427597,2673.610107,0.067859,0.04225,8175.0,21814.704895,15092.328747,2.668465
6,FB,2013-12-13,150,53.32,7998.0,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,19786.499359,4555.786405,176.460007,2673.610107,0.049700,0.04225,16173.0,49599.204254,27646.115152,3.066791
1,JNJ,2014-02-27,100,81.90,8190.0,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,...,4392.638550,4117.680233,130.557571,2673.610107,-0.036238,0.04225,24363.0,62181.842804,39953.795385,2.552306
2,MCD,2013-06-04,100,84.99,8499.0,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,...,6476.251770,6018.193382,162.893448,2673.610107,-0.080672,0.04225,32862.0,77157.094574,54470.988768,2.347912
3,MTCH,2015-12-14,600,13.63,8178.0,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,17399.618408,3092.646349,29.799627,2673.610107,0.430533,0.04225,41040.0,102734.712982,65741.635117,2.503282


In [70]:

# Referencing the adj_close dataframe from above

adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,68.502106
1,AAPL,2013-01-03,67.637436
2,AAPL,2013-01-04,65.753418
3,AAPL,2013-01-07,65.366646
4,AAPL,2013-01-08,65.542587


In [71]:
portfolio_df.head()


,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29
MCD,2013-06-04,100,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29


In [72]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace=True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
0,AAPL,2013-01-02,68.502106,2013-02-07,125,65.4,8175.0,2017-12-29
1,AAPL,2013-01-03,67.637436,2013-02-07,125,65.4,8175.0,2017-12-29
2,AAPL,2013-01-04,65.753418,2013-02-07,125,65.4,8175.0,2017-12-29
3,AAPL,2013-01-07,65.366646,2013-02-07,125,65.4,8175.0,2017-12-29
4,AAPL,2013-01-08,65.542587,2013-02-07,125,65.4,8175.0,2017-12-29


In [73]:

# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)

In [74]:

# Anything less than 0 means that the stock close was prior to acquisition.
adj_close_acq_date['Date Delta'] = adj_close_acq_date['Date'] - adj_close_acq_date['Acquisition Date']

adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)  

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
0,AAPL,2013-01-02,68.502106,2013-02-07,-3110400000000000
1,AAPL,2013-01-03,67.637436,2013-02-07,-3024000000000000
2,AAPL,2013-01-04,65.753418,2013-02-07,-2937600000000000
3,AAPL,2013-01-07,65.366646,2013-02-07,-2678400000000000
4,AAPL,2013-01-08,65.542587,2013-02-07,-2592000000000000


In [75]:

# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

adj_close_acq_date_modified.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
25,AAPL,2013-02-07,58.759972,2013-02-07,0
26,AAPL,2013-02-08,59.608299,2013-02-07,86400000000000
27,AAPL,2013-02-11,60.229511,2013-02-07,345600000000000
28,AAPL,2013-02-12,58.719791,2013-02-07,432000000000000
29,AAPL,2013-02-13,58.608109,2013-02-07,518400000000000


In [76]:

# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot

,Ticker,Acquisition Date,Adj Close
0,AAPL,2013-02-07,174.517639
1,FB,2013-12-13,193.089996
2,JNJ,2014-02-27,138.425430
3,MCD,2013-06-04,168.798950
4,MTCH,2015-12-14,42.629364
5,NFLX,2016-01-14,331.440002
6,TWTR,2015-01-05,52.869999
7,WMT,2013-08-14,103.966515


In [77]:

# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,AAPL,2013-02-07,174.517639,2018-03-09
1,FB,2013-12-13,193.089996,2018-02-01
2,JNJ,2014-02-27,138.425430,2018-01-22
3,MCD,2013-06-04,168.798950,2018-01-26
4,MTCH,2015-12-14,42.629364,2018-03-09


In [78]:

# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged
                                             , on=['Ticker', 'Acquisition Date'])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,174.517639,1.668465,1509.390015,...,2673.610107,0.067859,0.04225,8175.00,21814.704895,15092.328747,2.668465,174.517639,2018-03-09,0.000000
1,FB,2013-12-13,150,53.32,7998.00,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,2673.610107,0.049700,0.04225,16173.00,49599.204254,27646.115152,3.066791,193.089996,2018-02-01,-0.040706
2,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,125.826385,0.536342,1854.290039,...,2673.610107,-0.036238,0.04225,24363.00,62181.842804,39953.795385,2.552306,138.425430,2018-01-22,-0.091017
3,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,149.752518,0.762002,1631.380005,...,2673.610107,-0.080672,0.04225,32862.00,77157.094574,54470.988768,2.347912,168.798950,2018-01-26,-0.112835
4,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,2673.610107,0.430533,0.04225,41040.00,102734.712982,65741.635117,2.503282,42.629364,2018-03-09,0.000000
5,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,2673.610107,0.726610,0.04225,49193.25,127592.713165,77563.432360,2.593704,331.440002,2018-03-09,0.000000
6,TWTR,2015-01-05,225,36.38,8185.50,2017-12-29,2018-03-09,35.349998,-0.028312,2020.579956,...,2673.610107,0.472303,0.04225,57378.75,135546.462822,88852.007814,2.362311,52.869999,2015-04-07,-0.331379
7,WMT,2013-08-14,125,68.30,8537.50,2017-12-29,2018-03-09,84.700142,0.240119,1685.390015,...,2673.610107,-0.096213,0.04225,65916.25,146133.980560,102967.637877,2.216964,103.966515,2018-01-29,-0.185313


In [79]:

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['Share YTD'][0:10],
    name = 'Ticker YTD')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'][0:10],
    name = 'SP500 YTD')
    
data = [trace1, trace2]

layout = go.Layout(title = 'YTD Return vs S&P 500 YTD'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [80]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')
    
data = [trace1]

layout = go.Layout(title = 'Adj Close % off of High'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [81]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP Return'][0:10],
    name = 'SP500 Total Return')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker', tickformat=".2%")
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [82]:

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return ($)')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP 500 Gain / (Loss)'][0:10],
    name = 'SP 500 Total Return ($)')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return %',
    yaxis='y2')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Gain / (Loss) Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Gain / (Loss) ($)')
    , yaxis2=dict(title='Ticker Return', overlaying='y', side='right', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.75,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [83]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'],
    # mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum SP Returns'],
    # mode = 'lines+markers',
    name = 'Cum SP500 Returns')

trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'],
    # mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'],
    # mode = 'lines+markers',
    name = 'Cum ROI Mult'
    , yaxis='y2')


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.4,y=1)
    , yaxis2=dict(title='Cum ROI Mult', overlaying='y', side='right')               
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [86]:
merged_portfolio_sp_latest_YTD_sp_closing_high.to_csv('analyzed_portfolio.csv')

In [116]:
from pandas_datareader import data as pdr
import yfinance as yf
import datetime
yf.pdr_override() 
start = datetime.datetime(2020, 5, 23)
end = datetime.datetime(2020, 5, 25)
data = pdr.get_data_yahoo('AAPL',end)
data.set_index('ticker')
data
                          

[*********************100%***********************]  1 of 1 completed


KeyError: 'ticker'

In [123]:

a = datetime(2020,5,5)
s = 
s

TypeError: 'datetime.datetime' object is not subscriptable